In [1]:
import matplotlib.pyplot as plt
import torch
from PIL import Image
import torchvision.transforms.functional as TF
import os
import pandas as pd
import numpy as np
import json

In [2]:
import json
# Open a single json
directory = '0325updated.task2train(626p)'
filename = 'X51008164999.jpg'
with open('0325updated.task2train(626p)/X51008164999.txt') as json_file:
    data = json.load(json_file)
float(data['total'])
print(data)

# Open a single image and convert it to a tensor
image = Image.open(os.path.join(directory,filename))
image.show()
resized_im = image.resize((1024, 2048))
x = TF.to_tensor(resized_im)

{'company': 'ONE ONE THREE SEAFOOD RESTAURANT SDN BHD', 'date': '23-06-2018', 'address': 'NO.1, TAMAN SRI DENGKIL, JALAN AIR HITAM 43800 DENGKIL, SELANGOR.', 'total': '179.50'}


In [3]:
# Bad code to check how many images and how many txt files there are :p
# Without checking for repeated ones: 876 txt and 735 images 

count_images = 0
count_txt = 0
for filename in os.listdir(directory):
    if filename.endswith(".jpg") and not (filename.endswith("(1).jpg") or filename.endswith("(2).jpg" or \
                                    filename.endswith("(3).jpg") or filename.endswith("(4).jpg"))):
        count_images += 1
    elif filename.endswith(".txt") and not (filename.endswith("(1).txt") or filename.endswith("(2).txt" or \
                                    filename.endswith("(3).txt") or filename.endswith("(4).txt")) \
                                           or filename.endswith("(5).txt") or filename.endswith("(6).txt")):
        count_txt += 1
print("Number of images: {}\n Number of txt files: {} ".format(count_images,count_txt))
print("This doesn't quite work... How are there more txt files?")

Number of images: 632
 Number of txt files: 648 
This doesn't quite work... How are there more txt files?


In [4]:
# Find jpg and then find corresponding txt
image_name_jpg = 'X51005442376.jpg'
partitioned_string = filename.partition('.')
image_name_ = partitioned_string[0]
print(image_name_)
for filename in os.listdir(directory):
    if filename.startswith(image_name_) and filename.endswith('.txt'):
        print(filename)

X51006619697
X51006619697.txt


In [32]:
directory = '0325updated.task2train(626p)'

# First get the image -> then find the corresponding txt file O(n^2) 
images = np.empty(736, dtype = torch.Tensor)
labels = np.empty(736, dtype=torch.Tensor)
counter1 = 0
counter2 = 0
found_image = False
count = 0
for filename in os.listdir(directory):
    if filename.endswith(".jpg") and not found_image:
        partitioned_string = filename.partition('.')
        image_name = partitioned_string[0]
#         print(image_name, type(image_name))
#         print("Filename1: ", filename)
        image = Image.open(os.path.join(directory,filename))
        resized_im = image.resize((1024, 2048))
        # image.show()
        x = torch.flatten(TF.to_tensor(resized_im))
        found_image = True
        
    
    go_out = False
    if found_image: # Only look for txt file if we already have an image
        for filename2 in os.listdir(directory):
            if filename2.startswith(image_name) and filename2.endswith(".txt"):
                with open(os.path.join(directory,filename2)) as json_file:
        #                     data = json.load(json_file)
                    # print("Filename: ", filename, "Filename2", filename2, counter1)
                    images[counter1] = x
                    labels[counter1] = json.load(json_file)['total']
                    counter1 += 1
    #             found_image = False
                    go_out = True
                    found_image = False
            if go_out:
                break
        found_image = False # We couldn't find corresponding txt file

# 6291456 size of each torch

In [33]:
from re import sub
from decimal import Decimal

X_train = images.copy() # deep copy?
y_train = labels.copy()

print(X_train.shape, y_train.shape)

count_ = 0
indexes_to_remove = []
for ind, num in enumerate(y_train):
    if num == None:
        indexes_to_remove.append(ind)
    elif classify(num) == "float":
        y_train[ind] = float(num)
    else: 
        # We will remove it from both x_train and y_train since it is not a float...
        indexes_to_remove.append(ind)
#         y_train = np.delete(y_train, ind)
#         X_train = np.delete(X_train, ind)

print(indexes_to_remove)
y_train = np.delete(y_train, indexes_to_remove)
X_train = np.delete(X_train, indexes_to_remove)
print(X_train.shape, y_train.shape)

(736,) (736,)


NameError: name 'classify' is not defined

In [37]:
from re import sub
from decimal import Decimal

X_train = images.copy() # deep copy?
y_train = labels.copy()

for ind, num in enumerate(y_train):
    if num == None:
        indexes_to_remove.append(ind)
    else: 
        # We will remove it from both x_train and y_train since it is not a float...
        # print(num, type(num), len(num))
        if len(num) > 0:
            y_train[ind] = float(Decimal(sub(r'[^\d.]', '', num)))
        else:
            indexes_to_remove.append(ind)
                               
y_train = np.delete(y_train, indexes_to_remove)
X_train = np.delete(X_train, indexes_to_remove)
print(X_train.shape, y_train.shape)
print(y_train)

                               

(726,) (726,)
[7.6 12.0 72.0 8.2 36.0 119.7 3.7 8.7 5.9 329.1 25.15 41.87 31.45 35.4
 65.2 62.0 9.0 55.1 52.1 14.85 21.5 89.0 2.4 7.4 4.7 22.9 68.41 19.31 39.8
 12.72 60.6 98.35 7.3 27.1 12.0 15.9 8.7 31.8 12.0 5.9 23.2 247.55 3.9
 52.8 46.06 5.3 7.0 73.55 50.6 593.1 6.0 8.1 28.05 153.35 13.1 63.35 20.4
 83.0 21.6 29.3 9.0 14.0 45.0 8.7 16.0 2.0 31.45 32.7 19.4 29.68 40.0 18.0
 55.1 73.0 38.0 2.5 6.85 15.4 68.1 5.0 179.5 7.7 165.0 148.0 14.65 7.42
 142.0 23.56 18.9 23.25 10.2 53.3 12.4 2.5 6.0 39.0 25.6 5.9 6.0 248.05
 21.0 2.1 21.0 8.5 38.37 476.8 25.85 6.6 53.3 12.15 30.1 29.8 16.0 24.0
 8.7 39.9 29.3 27.2 758.7 42.4 33.8 42.4 7.7 6.7 7.4 62.8 48.15 73.0 18.9
 47.7 5.3 23.6 41.95 112.35 7.5 6.0 60.24 30.9 8.7 50.8 9.3 9.9 3.21 25.45
 8.9 10.3 51.42 7.4 7.0 7.7 7.97 102.0 10.3 41.2 9.9 93.07 8.6 46.9 24.0
 58.86 31.05 81.8 14.9 11.2 112.45 71.9 21.0 137.15 52.0 7.4 19.0 22.65
 51.3 101.75 28.5 71.0 78.1 79.35 9.65 21.2 39.75 116.28 23.0 35.0 12.0
 12.6 105.0 39.6 14.65 7.0 53.6 42.5 8

In [55]:
ind_to_remove = []
for ind, tensor_ in enumerate(X_train):
    if tensor_.size()[0] != 6291456:
        ind_to_remove.append(ind)
#         print(ind)
    
# print("Finished")
y_train = np.delete(y_train, ind_to_remove)
X_train = np.delete(X_train, ind_to_remove)

In [58]:
# print(y_train)
# from re import sub
# from decimal import Decimal

# money = 'RE$6,150,593.22'
# value = Decimal(sub(r'[^\d.]', '', money))
# print(value)
print(type(int(y_train[0])))
print(X_train.shape)

<class 'int'>
(723,)


In [59]:
# Idea: Create a Neural Network that gets the image and ouputs the total

class network(torch.nn.Module):
    
    def __init__(self):
        super(network, self).__init__()
        self.layer1 = torch.nn.Linear(in_features=6291456, out_features=40)
        self.layer2 = torch.nn.Linear(in_features=40, out_features=30)
        self.layer3 = torch.nn.Linear(in_features = 30, out_features=15)
        self.outputlayer = torch.nn.Linear(in_features=15, out_features=1)
        
    def forward(self,x):
        x = torch.relu(self.layer1(x))
        x = torch.relu(self.layer2(x))
        x = torch.relu(self.layer3(x))
        x = self.outputlayer(x)
        return x

In [61]:
learning_rate = 0.01
EPOCHS = 200
batch_size = 8
neural_network = network()

criterion = torch.nn.MSELoss()
optimizer = torch.optim.SGD(neural_network.parameters(), lr = learning_rate)

loss_in_epoch = []
accuracy_in_epoch = []
for epoch in range(EPOCHS):
    for X_train_row, y_train_row in zip(X_train, y_train):
        optimizer.zero_grad()
        output = neural_network(X_train_row)
        loss = criterion(output, torch.Tensor([y_train_row]))
        loss.backward()
        optimizer.step()
        
        
#     if (epoch % 10 == 0):
#         print()
#         loss_in_epoch.append(loss.item())
#         pred = torch.argmax(neural_network(torch.tensor(X_train)), dim=1).numpy()
#         accuracy_in_epoch.append(np.sum(pred == y_train)/len(y_train))
#         print(np.sum(pred == y_train)/len(y_train))

KeyboardInterrupt: 